Задание 1

    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [2]:
agg_ratings = ratings.groupby('userId').agg({'movieId':['count'],'timestamp':['min','max']})
agg_ratings = agg_ratings[agg_ratings['movieId']['count'] > 100]

from datetime import datetime as dt

def output_td(data):
    start = data['timestamp']['min']
    end = data['timestamp']['max']
    duration = dt.fromtimestamp(end) - dt.fromtimestamp(start)
    return str(duration)

agg_ratings['longevity'] = agg_ratings.apply(output_td, axis=1)
agg_ratings.head()

movieId   timestamp                        longevity
         count         min         max                     
userId                                                     
4          204   949778714   949982274      2 days, 8:32:40
8          116  1154389340  1154474527             23:39:47
15        1700   997937239  1469330735  5455 days, 22:38:16
17         363  1127468587  1127476640              2:14:13
19         423   855190091   855195373              1:28:02

Задание 2

    Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [3]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [4]:
result1 = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
result2 = result1.merge(client_base,on='client_id', how='left')
if not len(result1) == len(result2):
    raise Exception('wtf?')
result2.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


Задание 3

    В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

    У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
    Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
    Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Связать факт визита или покупки с набором координат можно по времени. 
По всей видимости, возможность снимать координаты пользователя была только во время его посещений, возможно не всех посещений, возможно не для всех пользователей.
Соответственно, визит может быть и без координат, а вот координат без визита быть не может.
Поэтому, для каждой снятой координаты мы можем сможем найти ближайший предшествующий ей визит и сопоставить их. Если окажется, что в день снятия координаты визита не было, то будем считать это ошибкой и не будем учитывать такие координаты.
Аналогичным образом можем сопоставить покупки с визитами - без визита нет покупки.
В результате получим табличку с визитами пользователей, где для некоторых визитов будут заполнены колонки с покупками и координатами.
Потом можно сопоставить даты визитов, по которым есть gps координаты, с рекламными кампаниями, которые проводились по этому географическому признаку в дни визитов, и, соответственно, проставить некоторым визитам признак конкретной рекламной кампании. 
Потом можно сравнивать, сколко покупок совершили пользователи, попавшие под рекламную кампанию, по сравнению с пользователями, не попавшими под рекламную кампанию, например, или еще как сравнивать...  Зависит от конкретной задачи.

Если координаты одного пользователя значительно удалены друг от друга, можно сделать вывод, что этот пользователь путешествует, или, что он купил товар в одном месте, для того, что бы пользоваться им в другом. Это можно использовать для более тщательного изучения поведения пользователей, для планиронваия новых рекламных кампаний в будущем.

Дополнительно, из координат можно узнать страну и населенный пункт, который пользователь посещал, или в котором он часто бывает . Так же можно узнать к какому типу климата относится это место, ВВП страны, уровень преступности, да все что угодно. Все это можно сделать дополнительными атрибутами в этом исследовании. Хотя есть сомнения, что на практике кто-то пользуется такими сложными моделями.

In [5]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)
import random
n = 10
users_with_gps = [11, 33, 44, 11, 99];
coordinates = pd.DataFrame(
    {
        'user_id': [users_with_gps[random.randint(0,4)] for i in range(n)],
        'latitude': [random.uniform(0,180) for i in range(n)],
        'longitude': [random.uniform(0,360) for i in range(n)]
    } 
)
coordinates.head()

,user_id,latitude,longitude
0,11,130.149822,226.807295
1,11,0.852991,242.867527
2,99,79.108385,87.089869
3,11,65.454550,346.911315
4,11,62.340808,99.158351
